# Overlap Correction with Linear Mixed Models (aka unmixed.jl)
### Benedikt Ehinger with help from Dave Kleinschmidt
### 2020-02-17

In [ ]:
using Revise
using CSV
using StatsModels
using MixedModels
using DataFrames
using DataFramesMeta
import Plots
import unfold
using Gadfly
using DataFramesMeta

Plots.gr()

This notebook is similar to the `lm_tutorial`, but fits mass-univariate *mixed* models and time-expanded/overlap-corrected *mixed* models.

## Reading input
The data were simulated in MatLab using the `unmixed toolbox (www.unfoldtoolbox.org)` with the function`EEG_to_csv.m`.

**Limitation**: due to current implementation in MixedModels.jl, we cannot fit overlap-corrected random effects.
That is, the `(1|item)` cannot be modelled at the moment.

In [ ]:
testCase = "testCaseMultisubject2"
data = CSV.read("test\\"*testCase*"_data.csv", header=0)
data = convert(Array,data)
evts = CSV.read("test\\"*testCase*"_events.csv")
categorical!(evts,:subject);

The `events` dataFrame looks like this

In [ ]:
first(evts,6)

With the important fields being `latency`, `condA`, `condB` and `subject`.

The data are a vector.

In [ ]:
println(typeof(data))
println(size(data))

**Limitation** Note how small it is! Only 12k samples, that is only ~5minutes of recording in total for 25 subjects. More realistic samples quickly take hours to fit.

## Without Overlap Correction
We define the formula

In [ ]:
f  = @formula 0~1+condA*condB+(1+condA*condB|subject);

epoch the data once again

In [ ]:
data_r = reshape(data,(1,:))
# cut the data into epochs
data_epochs,times = unfold.epoch(data=data_r,tbl=evts,τ=(-0.4,0.8),sfreq=50);

We can now run the LinearMixedModel on each time point

In [ ]:
m = unfold.fit(unfold.UnfoldLinearMixedModel,f,evts,data_epochs,times)

And plot it:

In [ ]:
d = @linq m.results |> where(:group.=="fixed")
plot(d,x=:time,y=:estimate,color=:term,Geom.LineGeometry)

And the random effect results

In [ ]:
d = @linq m.results |> where(:group.=="subject")
plot(d,x=:time,y=:estimate,color=:term,Geom.LineGeometry)

Obviously there is still something wrong with the random effects - but we are getting there :-D

## With Overlap Correction
For overlap correction, we have to use a basis function once again.

In [ ]:
basisfunction = unfold.firbasis(τ=(-0.05,.4),sfreq=40)
f  = @formula 0~1+condA*condB+(1+condA*condB|subject);

**Limitation:** Currently we cannot model correlation between time-points or random slopes.

**Limitation:** See the low sampling frequency? This is because the modelfit increases quadratically with the number of predictors

We can now run the mixed model.

Easy syntax: Specify formula, events, EEG-data & the basis function

In [ ]:
@time mm = unfold.fit(unfold.UnfoldLinearMixedModel,f,evts,data,basisfunction)

We receive an object containing the mixed model:

In [ ]:
show(coeftable(mm.model))

The VarCor - Random effect matrix is very large, so we are not showing it here.

But we also get a *tidy*-dataframe with the results

In [ ]:
first(mm.results,6)

and thus we can easily plot the fixed effect results.

In [ ]:
d = @linq mm.results |> where(:group.=="fixed")
plot(d,x=:time,y=:estimate,color=:term,Geom.LineGeometry)

And the random effect results.

In [ ]:
d = @linq mm.results |> where(:group.=="subject")
plot(d,x=:time,y=:estimate,color=:term,Geom.LineGeometry)

## What is happening under the hood?

In [ ]:
Xs,form = unfold.LinearMixedModel_formula(f,evts,basisfunction, contrasts = Dict{Symbol,Any}());

Formula-Terms are wrapped with a `TimeExpandedTerm`, which upon calling `modelcols` will timeexpand the designmatrix.
There is one TimeExpandedTerm for the FixedEffects and one for each RandomEffectsTerm.

 **Limitation** Currently it is not possible to combine different formulas & different basis-functions, but that is planned

In [ ]:
typeof(form.rhs)

Visualizing the designmatrices.
Fixed Effects:

In [ ]:
Plots.heatmap(Matrix(Xs[1][1:300,:]))

Random Effects

In [ ]:
Plots.heatmap(Matrix(Xs[2][1:2000,1:500]))

And finally, generate the linear mixed model manually & fit it.

In [ ]:
mm = unfold.LinearMixedModel_wrapper(form,data,Xs)
fit!(mm)
mm = unfold.condense(mm,evts)

## Summary
There are four different model types currently "fitable"

1. Timeexpansion **No**, Mixed **No**  : `unfold.fit(unfold.UnfoldLinearModel,f,evts,data_epoch,times)`
1. Timeexpansion **Yes**, Mixed **No** : `unfold.fit(unfold.UnfoldLinearModel,f,evts,data,basisfunction)`
1. Timeexpansion **No**, Mixed **Yes** : `unfold.fit(unfold.UnfoldLinearMixedModel,f,evts,data_epoch,times)`
1. Timeexpansion **Yes**, Mixed **Yes**: `unfold.fit(unfold.UnfoldLinearMixedModel,f,evts,data,basisfunction)`

In [ ]:
using Weave

# convert to html
weave("doc\\lmm_tutorial.jmd")

# convert to a python notebook
convert_doc("doc\\lmm_tutorial.jmd", "doc\\lmm_tutorial.ipynb")

# convert to md for README
#weave("doc\\lm_tutorial.jmd", doctype="pandoc", out_path = "README.md")